In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [3]:
from multiversx_sdk_core import Address, Transaction, TransactionPayload
from multiversx_sdk_core.transaction_builders import DefaultTransactionBuildersConfiguration
from multiversx_sdk_core import TokenPayment
from multiversx_sdk_network_providers import ProxyNetworkProvider, ApiNetworkProvider
from tools.notebooks import env
import config

builder_config = DefaultTransactionBuildersConfiguration(chain_id="1")
proxy_mainnet = ProxyNetworkProvider("https://gateway.multiversx.com")
api_mainnet = ApiNetworkProvider("https://api.multiversx.com")
proxy_shadowfork = ProxyNetworkProvider(config.DEFAULT_PROXY)

In [4]:
from multiversx_sdk_core import AccountNonceHolder
from multiversx_sdk_wallet import UserSigner
from pathlib import Path

user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))

In [5]:
def prep_forward_tx(tx_hash: str) -> Transaction:
    original_tx_data = proxy_mainnet.get_transaction(tx_hash)
    print(original_tx_data.data)
    forwarded_tx  = Transaction(
        sender=original_tx_data.sender,
        receiver=original_tx_data.receiver,
        value=TokenPayment.egld_from_integer(original_tx_data.value),
        data=TransactionPayload.from_str(original_tx_data.data),
        chain_id=builder_config.chain_id,
        gas_limit=original_tx_data.gas_limit,
        gas_price=builder_config.min_gas_price
    )

    print(forwarded_tx.to_dictionary())


def sign_and_send(tx: Transaction, account: UserSigner):
    user_nonce_holder = AccountNonceHolder(proxy_shadowfork.get_account(tx.sender).nonce)
    tx.nonce = user_nonce_holder.get_nonce_then_increment()
    tx.signature = account.sign(tx)
    proxy_shadowfork.send_transaction(tx)

clone & forward transaction

In [ ]:
hash = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

forwarded_tx = prep_forward_tx(hash)

In [ ]:
sign_and_send(forwarded_tx, user_signer)

clone & forward transactions on account

In [6]:
from time import sleep
contract_address = "erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg"

transactions = api_mainnet.get_transactions(Address.from_bech32(contract_address))
for transaction in transactions:
    print(f"Forwarded tx hash: {transaction.hash}")
    forwarded_tx = prep_forward_tx(transaction.hash)
    # sign_and_send(forwarded_tx, user_signer)
    # sleep(1)

ESDTNFTTransfer@45474c444d4558464c2d633235323165@050eb7@360ed7b8458afb4b@00000000000000000500e84cd0951232d1a7ee4ebff319eae738b172e7d6b055@636c61696d52657761726473
OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1jqryzwzwr5rqh52c3gca40a6ervpt4gp8wfknlvqyrrd2zuyh7dsj3vxxn'), ('sender', 'erd1jqryzwzwr5rqh52c3gca40a6ervpt4gp8wfknlvqyrrd2zuyh7dsj3vxxn'), ('gasPrice', 1000000000), ('gasLimit', 22000000), ('data', 'RVNEVE5GVFRyYW5zZmVyQDQ1NDc0YzQ0NGQ0NTU4NDY0YzJkNjMzMjM1MzIzMTY1QDA1MGViN0AzNjBlZDdiODQ1OGFmYjRiQDAwMDAwMDAwMDAwMDAwMDAwNTAwZTg0Y2QwOTUxMjMyZDFhN2VlNGViZmYzMTllYWU3MzhiMTcyZTdkNmIwNTVANjM2YzYxNjk2ZDUyNjU3NzYxNzI2NDcz'), ('chainID', '1'), ('version', 1), ('signature', '')])
4f94be25fcea64e04c88e4b466c9f441de289e7b5c4b2b0b97652e3fd2410dea
ESDTNFTTransfer@45474c444d4558464c2d633235323165@04f648@0d396f7899d8032b@00000000000000000500e84cd0951232d1a7ee4ebff319eae738b172e7d6b055@636c61696d52657761726473
OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1wg22xhlm0hug4

KeyboardInterrupt: 